In [20]:
import pandas as pd
import numpy as np
import string as str
from os import listdir
from os.path import isfile, join
from  datetime import datetime
from os import listdir
from os.path import isfile, join

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [11]:
#single file
path='/home/ayush/Desktop/BigData_ProjectData/PlayStoreCsv/data/big_data2.csv'
df= pd.read_csv(path,sep=',',header=0)

#whole data
'''dfs=[]
path='../data/'
onlyfiles = [ f for f in listdir(path) if isfile(join(path,f)) and f.endswith( 'csv' ) ]
print onlyfiles
for eachFile in onlyfiles:
    path1='../data/'+eachFile
    df1= pd.read_csv(path1,sep=',',header=0)
    print df1.shape[0]
    dfs.append(df1)
df=pd.concat(dfs)
df.shape[0]'''

"dfs=[]\npath='../data/'\nonlyfiles = [ f for f in listdir(path) if isfile(join(path,f)) and f.endswith( 'csv' ) ]\nprint onlyfiles\nfor eachFile in onlyfiles:\n    path1='../data/'+eachFile\n    df1= pd.read_csv(path1,sep=',',header=0)\n    print df1.shape[0]\n    dfs.append(df1)\ndf=pd.concat(dfs)\ndf.shape[0]"

In [12]:
def mapBooleanColumns(x):
    if x==True:
        return 1
    else:
        return 0
    
def mappingScores(x):
    if x>0 and x<=1:
        return 1
    elif x>1 and x<=2:
        return 2
    elif x>2 and x<=3:
        return 3
    elif x>3 and x<=4:
        return 4
    else:
        return 5

def mapDates(d):
    d=d[:10]
    d1= datetime.now()
    d2=datetime.strptime(d, "%Y-%m-%d")
    return (d1.year - d2.year)*12 + d1.month - d2.month    
    
def seperateInstallations(col_name):
    return_data_type_as=None
    ls = col_name.str.split('-').str.get(0).str.strip(' ').str.replace(',','') #series object
    hs = col_name.str.split('-').str.get(1).str.strip(' ').str.replace(',','') #series object
    ls = ls.astype(float).fillna(0.0)
    hs = hs.astype(float).fillna(0.0)
    return ls, hs

def processAppSize(x):
    if x==-1:
        return 0.5
    else:
        return x

In [13]:
# Transforming the data into numerical forms
df.IsTopDeveloper=map(mapBooleanColumns,df.IsTopDeveloper)
df.HaveInAppPurchases=map(mapBooleanColumns,df.HaveInAppPurchases)
df.IsFree=map(mapBooleanColumns,df.IsFree)
df.Score=map(mappingScores,df.Score)
df.PublicationDate=map(mapDates,df.PublicationDate)
df.LastUpdateDate=map(mapDates,df.LastUpdateDate)
df['minInstall'],df['maxInstall']=seperateInstallations(df.Instalations)
df.AppSize=map(processAppSize,df.AppSize)

In [30]:
# Selecting Features
df=df.head(n=2000)
X=df[['Price','PublicationDate','AppSize','LastUpdateDate','IsFree','HaveInAppPurchases','minInstall','maxInstall']]
y=pd.DataFrame(df[['Score']])

In [31]:
# Vectorising the text features
import nltk.stem
s= nltk.stem.SnowballStemmer('english')
s.stem("graphics")
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        english_stemmer = nltk.stem.SnowballStemmer('english')
        analyzer = super(StemmedCountVectorizer,self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))
    
def vectorize_column(dataframe,column_name,vectorizer=None):
    if vectorizer is None:
        print("No Vectorizer is explicitly specified. Using CountVectorizer as default one. ")
        column_vectorizer = CountVectorizer(min_df=1)
    else:
        column_vectorizer = vectorizer
    if column_name in dataframe.columns.values.tolist():
        column_df = dataframe[column_name]
        fmatrix = column_vectorizer.fit_transform(column_df)
        print column_vectorizer.get_feature_names()
        #print("vectorized into matrix of shape"), fmatrix.toarray().shape
        dataframe_f = pd.DataFrame(fmatrix.toarray(), columns=column_vectorizer.get_feature_names())
        print("formed dataframe of size:("),dataframe_f.index.max()+1,",", dataframe_f.head(1).shape[1],")"   
        return dataframe_f, fmatrix, column_vectorizer
    else:
        print("No column found")  
    
stem_vectorizer = StemmedCountVectorizer(min_df =1, stop_words='english')
# Vectorizing the Category Column
newfeature, fmatrix, column_vectorizer = vectorize_column(df, 'Category', stem_vectorizer)
# Vectorizing the Name Column
newfeature1, fmatrix1, column_vectorizer1 = vectorize_column(df, 'Name', stem_vectorizer)
# Vectorizing the Description Column
newfeature2, fmatrix2, column_vectorizer2 = vectorize_column(df, 'Description', stem_vectorizer)
# Vectorizing the Content Rating Column
newfeature3, fmatrix3, column_vectorizer3 = vectorize_column(df, 'ContentRating', stem_vectorizer)


[u'books_and_refer', u'busi', u'comic', u'communic', u'educ', u'entertain', u'financ', u'game_act', u'game_adventur', u'game_arcad', u'game_board', u'game_card', u'game_casu', u'game_educ', u'game_puzzl', u'game_rac', u'game_sport', u'game_trivia', u'health_and_fit', u'libraries_and_demo', u'lifestyl', u'media_and_video', u'medic', u'music_and_audio', u'news_and_magazin', u'person', u'photographi', u'product', u'shop', u'social', u'sport', u'tool', u'transport', u'travel_and_loc', u'weather']
formed dataframe of size:( 1000 , 35 )
[u'100', u'101', u'14', u'16', u'18', u'2014', u'2015', u'2048', u'21', u'24h', u'2d', u'310', u'33', u'3d', u'3g', u'46', u'5color', u'9cube', u'aarti', u'abc', u'acceler', u'accelerometerplay', u'accept', u'access', u'accur', u'activ', u'acurr', u'ad', u'address', u'advanc', u'air', u'aircraft', u'aka', u'al', u'alarm', u'album', u'alert', u'allah', u'alphabet', u'altimet', u'altitud', u'amidatsunagi', u'amp', u'anagog', u'android', u'anim', u'anita', u'ann

In [32]:
print df.shape
print X.shape
print X.columns
print newfeature.shape
print newfeature.columns

finaldf = pd.concat([X,newfeature,newfeature1],axis=1)

print finaldf.shape
print y.shape

(1000, 17)
(1000, 8)
Index([u'Price', u'PublicationDate', u'AppSize', u'LastUpdateDate', u'IsFree',
       u'HaveInAppPurchases', u'minInstall', u'maxInstall'],
      dtype='object')
(1000, 35)
Index([   u'books_and_refer',               u'busi',              u'comic',
                 u'communic',               u'educ',          u'entertain',
                   u'financ',           u'game_act',      u'game_adventur',
               u'game_arcad',         u'game_board',          u'game_card',
                u'game_casu',          u'game_educ',         u'game_puzzl',
                 u'game_rac',         u'game_sport',        u'game_trivia',
           u'health_and_fit', u'libraries_and_demo',           u'lifestyl',
          u'media_and_video',              u'medic',    u'music_and_audio',
         u'news_and_magazin',             u'person',        u'photographi',
                  u'product',               u'shop',             u'social',
                    u'sport',               u'

In [33]:
n_components=[100,200,500,1000]
classifiers = [GaussianNB(),BernoulliNB(),RandomForestClassifier(),LogisticRegression(),SGDClassifier(),RidgeClassifier()
              ,DecisionTreeClassifier()]
classifiername=[Gaussian,Bernauli,RForest,LRegression,SDG,Ridge,DecisionTree]
for n in n_components:
    xx=finaldf.copy()
    pca = PCA(n_components=n)
    xx= pca.fit_transform(xx)
    print xx.shape
    for clf in classifiers:
            X_train, X_test, y_train, y_test = train_test_split(xx, y.values, test_size=.4)
            clf.fit(X_train,y_train)
            y_pred=clf.predict(X_test)
            print("Components:%f Classifier:%s Accuracy Score:%f"%(n,clf,accuracy_score(y_test,y_pred)))
            #print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0],(y_test != y_pred).sum()))

(1000, 100)
Components:100.000000 Classifier:GaussianNB() Accuracy Score:0.325000
Components:100.000000 Classifier:BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) Accuracy Score:0.490000
Components:100.000000 Classifier:RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False) Accuracy Score:0.567500

/home/ayush/Documents/big-data/bigdata-env/local/lib/python2.7/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ayush/Documents/big-data/bigdata-env/local/lib/python2.7/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ayush/Documents/big-data/bigdata-env/lib/python2.7/site-packages/ipykernel/__main__.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/ayush/Documents/big-data/bigdata-env/local/lib/python2.7/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y


Components:100.000000 Classifier:LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0) Accuracy Score:0.542500
Components:100.000000 Classifier:SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False) Accuracy Score:0.115000
Components:100.000000 Classifier:RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, solver='auto', tol=0.001) Accuracy Score:0.555000
Components:100.000000 Classifier:DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_n

/home/ayush/Documents/big-data/bigdata-env/local/lib/python2.7/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ayush/Documents/big-data/bigdata-env/local/lib/python2.7/site-packages/sklearn/linear_model/ridge.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ayush/Documents/big-data/bigdata-env/local/lib/python2.7/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Components:200.000000 Classifier:GaussianNB() Accuracy Score:0.405000
Components:200.000000 Classifier:BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) Accuracy Score:0.542500
Components:200.000000 Classifier:RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False) Accuracy Score:0.500000

...

/home/ayush/Documents/big-data/bigdata-env/local/lib/python2.7/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ayush/Documents/big-data/bigdata-env/lib/python2.7/site-packages/ipykernel/__main__.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Components:1000.000000 Classifier:RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False) Accuracy Score:0.537500
Components:1000.000000 Classifier:LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0) Accuracy Score:0.540000

/home/ayush/Documents/big-data/bigdata-env/local/lib/python2.7/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ayush/Documents/big-data/bigdata-env/local/lib/python2.7/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Components:1000.000000 Classifier:SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False) Accuracy Score:0.407500
Components:1000.000000 Classifier:RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, solver='auto', tol=0.001) Accuracy Score:0.522500
Components:1000.000000 Classifier:DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best') Accuracy Score:0.502500


/home/ayush/Documents/big-data/bigdata-env/local/lib/python2.7/site-packages/sklearn/linear_model/ridge.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
